In [1]:
import pandas as pd
import numpy as np

In [2]:
def test_p(nodes, connections):
    neighbour_nodes = np.roll(nodes, -1)
    return map(lambda v: connections[tuple(v)],
               np.column_stack([nodes, neighbour_nodes]))


def find_cycle_enumerate(nodes, connections):
    solutions = []
    for p in itertools.permutations(nodes[1:], 4):
        order = (nodes[0], ) + p
        if sum(test_p(order, connections)) == nodes.size:
            solutions.append(order)
    return solutions
#     raise ValueError('No cycle exists between nodes.')

In [3]:
# %timeit find_cycle_enumerate(nodes_i, connections)

In [4]:
def find_cycle(nodes, connections, starting_temperature=1):
    nodes_i = np.array(nodes, dtype=int)
    score_i = sum(test_p(nodes_i, connections))
    if score_i >= len(nodes_i):
        return nodes_i
    temperature = starting_temperature
    
    for i in xrange(100):
        swaps_evaluated = 0
        swaps_accepted = 0

        for j in xrange(100):
            source, target = np.random.randint(1, len(nodes_i), size=2)
            nodes_j = nodes_i.copy()
            nodes_j[[source, target]] = nodes_i[[target, source]]
            score_j = sum(test_p(nodes_j, connections))
            if score_j >= len(nodes):
                return nodes_j

            keep_anyway = (np.random.rand() <
                           np.e ** (.5 * (score_j - score_i) / temperature))

            if score_j >= score_i or keep_anyway:
                nodes_i = nodes_j
                score_i = score_j
                swaps_accepted += 1
            swaps_evaluated += 1

        success_ratio = swaps_accepted / float(swaps_evaluated)

        if success_ratio > .96:
            temperature *= .5
        elif success_ratio > .8:
            temperature *= .9
        elif success_ratio > .15:
            temperature *= .95
        else:
            temperature *= .8
        if i % 10 == 0:
            print 'temperature: %s, success: %.2f' % (temperature,
                                                      success_ratio)
    raise ValueError('No cycle found (score: %s) %s' %
                     (score_i, nodes_i))

In [5]:
import itertools

# N = 15
N = 20

nodes = np.arange(N, dtype=int)
connections = np.zeros((nodes.size, nodes.size), dtype=int)

for i in xrange(nodes.size - 1):
    connections[i, i + 1] = 1
    connections[i + 1, i] = 1
connections[0, nodes.size - 1] = 1
connections[nodes.size - 1, 0] = 1

In [6]:
from datetime import datetime
nodes_i = nodes.copy()
np.random.shuffle(nodes_i[1:])

start = datetime.now()
result = find_cycle(nodes_i, connections, len(nodes_i) ** 1.5)
print nodes_i
print result
(datetime.now() - start).total_seconds()

temperature: 44.72135955, success: 1.00
temperature: 0.871081243772, success: 0.84
temperature: 0.443456911411, success: 0.53
temperature: 0.265514033822, success: 0.24
temperature: 0.133872050472, success: 0.15
temperature: 0.0170696023342, success: 0.03
temperature: 0.00183283459453, success: 0.09
temperature: 0.000196799116062, success: 0.08
temperature: 2.11311441842e-05, success: 0.06
temperature: 2.26893932995e-06, success: 0.07


ValueError: No cycle found (score: 14) [ 0  1  9  8  7  6  2  3  4  5 10 11 15 16 17 18 14 13 12 19]